# 流量を管理するモジュール  

本モジュールで計算できる流量は2018/10/9現在以下の3つである。

- $q_s$ 熱流（固体熱伝導）[J/m2s]
- $j_v$ 水蒸気流 [kg/m2s]
- $j_l$ 液水流 [kg/m2s]
  

## 1. 流量の計算方法


### 1.1 熱伝導（フーリエの法則）  
定義：$\dot q = -\lambda\nabla・T$  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\lambda$：熱伝導率[W/mK]  
$T$：絶対温度[K]  

In [1]:
def cal_heat_conduction( lam, dtemp, dx2 ):
    return - lam * dtemp / dx2

#### 差分化その1：$\dot q = -\lambda_{ave.}\frac{T_{l+1}-T_{l}}{dx_{l+1}+dx_{l}}$  
$\lambda_{ave.}$：平均の熱伝導率  
$l, l+1$：ある点のセル・隣接する点のセル  
$dx$：セルの質点からセル境界までの距離

In [2]:
def cal_heat_conduction_v1( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    lamAve = mean_average( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - lamAve * ( tempL1 - tempL ) / ( dxL1 + dxL )

#### 差分化その2：$\dot q = -K(T_{l+1}-T_{l})$  
$K$：熱貫流率[W/K]  
一般的にはこちらを採用。異種材料間の熱の移動は熱抵抗値の和として計算。

In [3]:
def cal_heat_conduction_v2( lamL1, lamL, tempL1, tempL, dxL1, dxL ):
    K = transmittance( val_mns = lamL, val_pls = lamL1, len_mns = dxL, len_pls = dxL1 )
    return - K * ( tempL1 - tempL )

### 熱伝達・熱貫流  

定義：$\dot q = \alpha(T_{air} - T_{wall})$  
空気に接する壁表面には空気の熱伝達層が存在すると考えた場合、もしくは質点間の熱貫流率として計算した場合の熱流。  
ここでは便宜上、空気と壁表面の熱伝達を想定し、空気から壁表面への熱流を正とした。

$\dot q$：単位時間当たりの熱流量の密度[W/m2]  ]
$\alpha$：熱伝達率・熱貫流率[W/m2K]  
$T_{air}$：空気の絶対温度[K]  
$T_{wall}$：壁表面の絶対温度[K]  

In [4]:
def cal_heat_transfer( alpha, temp_air, temp_wall ):
    return alpha * ( temp_air - temp_wall )


<br />

### 1.2 水蒸気移動  
元原理：Fickの拡散法則

#### 重力加速度

In [5]:
def grav():
    return 9.806650

### 水蒸気移動（水蒸気圧勾配・固体内）  
定義：$J_v = -\lambda^{'}_{p}\nabla・P_v$  
$J_v$：気相水分流量[kg/m2s]  
$\lambda^{'}_{p}$：水蒸気圧勾配に関する気相水分伝導率[kg/msPa]  
$P_v$：水蒸気圧[Pa]  

In [6]:
def cal_vapour_permeance_pressure( dp, dpv, dx2 ):
    return - dp * dpv /dx2

### 湿気伝達・湿気貫流  
定義：$J_v = \alpha^{'}_{m}(P_{v_{air}} - P_{v_{wall}})$  
空気に接する壁表面には空気の湿気伝達層が存在すると考えた場合の水分流（水蒸気流）、もしくは質点間の湿気貫流率として計算した場合の水分流。  
ここでは便宜上、空気と壁表面の水分伝達を想定し、空気から壁表面への水分流を正とした。

$J_v$：単位時間当たりの水分流量[kg/m2s]  
$\alpha^{'}_{m}$：湿気伝達率[kg/m2s(Pa)]  
$P_{v_{air}}$：空気の水蒸気圧[Pa]  
$P_{v_{wall}}$：壁表面の水蒸気圧[Pa]  

In [7]:
def cal_vapour_transfer_pressure( aldm, pv_air, pv_wall ):
    return aldm * ( pv_air - pv_wall )


<br />

### 1.3 液水移動  
原理：ダルシー則

### 液水移動（水分化学ポテンシャル勾配・固体内）  
定義：$J_l = -\lambda^{'}_{\mu_l}(\nabla・\mu - n_x g)$    

$J_l$：液相水分流量[kg/m2s]  
$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に関する液相水分伝導率[kg/ms(J/kg)]  
$\mu$：液水の化学ポテンシャル[J/kg]  
$n_x$：重力加速度に対する応答（重力加速度に対して水平なら1、垂直なら0）

In [8]:
def cal_liquid_conduction_potential( ldml, dmiu, dx2, nx ):
    return - ldml * ( dmiu / dx2 - nx * grav() )


<br />
<br />

### 補足：物性値の平均化   
セル間の物性値を計算するにあたり2つのセルの物性値を平均化する必要がある。  
この際、平均化の方法についてはいくつかやり方が存在するがここでは以下の2つの方法について示す。

#### 抵抗値として重ね合わせる  
セル間の流量が一定の場合、その間の物性値は各セルの物性値の抵抗値の和として表すことができる。したがって、  

熱伝導抵抗：  $R[m^2K/W] =\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} }  $  

同様に水分に関しても抵抗値の和として計算することで物性値を平均化することができる。  
ここで熱伝導率・水分伝導率を合わせて$\lambda$と表記すると、平均の伝導率は以下のように表すことができる。  

平均の伝導率：  $\lambda =\frac{ dx_{l+1} + dx_{l} }{\frac{ dx_{l+1} } { \lambda_{l+1} } + \frac{ dx_{l} }{ \lambda_{l} } } $  


In [9]:
def sum_resistance( val_mns, val_pls, len_mns, len_pls ):
    return (len_mns + len_pls) / ( len_mns / val_mns + len_pls / val_pls )

In [10]:
# 使用例
sum_resistance( val_mns = 10.0, val_pls = 1.0, len_mns = 0.1, len_pls = 0.1 )

1.8181818181818183

#### ※片方が空気の場合  
材料が空気と接する場合、空気から材料表面への熱移動は熱伝達及び放射により生じ、材料中から材料表面までは熱伝導によって生じる。  
このような場合、材料表面に質点を置くことが一般的であるが、質点の位置をCellクラスに入れることは非常に手間がかかる。  
そのためここでは空気から材料中への熱流が定常状態であるとし、熱抵抗の和として計算を行うこととする。   
空気および材料の熱貫流率は以下の式で表される。  

熱貫流率：$K =\frac{ 1 }{ \frac{ 1 }{ \alpha } + \frac{ dx_{l} }{ 2\lambda_{l} } } $  


In [11]:
def cal_transmittance( alpha, lam, dx ):
    return ( 1.0 ) / ( 1.0 / alpha + dx / ( 2.0 * lam ) )

#### 調和平均を取る  
一方でセルの長さに応じた物性値の調和平均を取るという考え方もある。
この平均化の仕方は伝導率の差が小さい時は問題ないが、差が大きくなると抵抗値の考え方と大きくずれてくる。

物性値の調和平均：　　$\lambda_{ave}=\frac{\lambda_{l+1} dx_{l+1} + \lambda_{l} dx_{l} }{ dx_{l+1} + dx_{l} }$  


In [12]:
def cal_mean_average( val_mns, val_pls, len_mns, len_pls ):
    return ( val_mns * len_mns + val_pls * len_pls ) / ( len_mns + len_pls )

In [14]:
# 使用例
cal_mean_average( val_mns = 10.0, val_pls = 1.0, len_mns = 0.1, len_pls = 0.1 )

5.5


<br />
<br />

## 2. Cellクラスを用いて計算を行う場合  
流量の計算を行うにあたり、上位のクラスにCellクラスと呼ばれる気相・液相・固相の状態量を保持したクラスを作成しておくと計算が簡略化できる。  
ここでは水蒸気（vapour）、液水（water）、材料骨格部（material）がそれぞれ保持されたCellクラスを用いた計算方法について示す。  
なお流量の計算式については1節の関数を用い簡略化した。  


注意点：

- 流量を計算するにあたり、材料間の質点の距離、物性値の平均値、セル間の重力に対する応答方向などを取る必要がある。
- 特に質点間の距離については質点の設定という工程を省略するため、ここでは全てのセルの質点を中央に取ることにより質点間の距離を簡単に計算できる方式を採用した。  
- そのため空気と接する材料境界では熱及び水分の移動は、"空気から材料表面のへの熱・水分流　＋　材料表面からの材料の質点までの熱・水分流 "とした
- 材料表面から空気への水分流は水蒸気のみになるとし、液相水分流は生じないものとした。

#### 固体熱伝導

In [26]:
def cal_heat_conduction_byCell(cell_mns, cell_pls):
    return cal_heat_conduction(lam = sum_resistance( val_mns=cell_mns.lam, val_pls=cell_pls.lam, len_mns=cell_mns.dx, len_pls=cell_pls.dx ),\
                               dtemp= cell_pls.material.temp - cell_mns.material.temp,\
                               dx2  = cell_mns.dx / 2.0 + cell_pls.dx / 2.0 )

#### 熱伝達

In [27]:
def cal_heat_transfer_byCell( cell_mns, cell_pls ):
    # mns側が空気である場合（mnsのセルにairがセットされている場合）
    if hasattr(cell_mns, 'air'):
        return cal_heat_transfer( alpha  = cal_transmittance( cell_mns.alpha, cell_pls.lam, cell_pls.dx ),\
                                 temp_air = cell_mns.air.temp,\
                                 temp_wall= cell_pls.material.temp )
    # pls側が空気である場合（plsのセルにairがセットされている場合）
    elif hasattr(cell_pls, 'air'):
        return - cal_heat_transfer( alpha  = cal_transmittance( cell_pls.alpha, cell_mns.lam, cell_mns.dx ),\
                                   temp_air = cell_pls.air.temp,\
                                   temp_wall= cell_mns.material.temp )

#### 水蒸気拡散（水蒸気圧勾配・固体内）

In [28]:
def cal_vapour_permeance_pressure_byCell( cell_mns, cell_pls, nx_dir ):
    return cal_vapour_permeance_pressure( dp = sum_resistance( val_mns=cell_mns.dp, val_pls=cell_pls.dp, len_mns=cell_mns.dx, len_pls=cell_pls.dx ),\
                                         dpv = cell_pls.vapour.pv - cell_mns.vapour.pv,\
                                         dx2 = cell_mns.dx / 2.0 + cell_pls.dx / 2.0 )

#### 対流による水蒸気移動

In [29]:
def cal_vapour_transfer_pressure_byCell( cell_mns, cell_pls, nx ):
    # mns側が空気である場合（mnsのセルにairがセットされている場合）
    if hasattr(cell_mns, 'air'):
        return cal_vapour_transfer_pressure( aldm = cal_transmittance( cell_mns.aldm, cell_pls.dp, cell_pls.dx ),\
                                            pv_air = cell_mns.air.pv,\
                                            pv_wall= cell_pls.vapour.pv )
    # pls側が空気である場合（plsのセルにairがセットされている場合）
    elif hasattr(cell_pls, 'air'):
        return - cal_vapour_transfer_pressure( aldm = cal_transmittance( cell_pls.aldm, cell_mns.dp, cell_mns.dx ),\
                                              pv_air = cell_pls.air.pv,\
                                              pv_wall= cell_mns.vapour.pv )

#### 液水移動

In [19]:
def cal_liquid_conduction_potential_byCell( cell_mns, cell_pls, nx_dir ):
    return cal_liquid_conduction_potential(ldml= sum_resistance( val_mns=cell_mns.ldml, val_pls=cell_pls.ldml, len_mns=cell_mns.dx, len_pls=cell_pls.dx ),\
                                           dmiu = self.cell_pls.water.miu - self.cell_mns.water.miu,\
                                           dx2  = cell_mns.dx / 2.0 + cell_pls.dx / 2.0,\
                                           nx   = nx_dir )


<br />
<br />

## 3. 境界面を自動的に判別する場合  
2節を応用し、材料の境界面を自動的に判別し、用いる流量計算の式を自動的に変更する式について示す。

#### 熱移動

In [30]:
def cal_qs(cell_mns, cell_pls ):
    # 両方のセルがともに多孔質材料である場合
    if hasattr(cell_mns, 'material') and hasattr(cell_pls, 'material'):
        return cal_heat_conduction_byCell( cell_mns, cell_pls )
    # 一方が空気境界である場合
    elif hasattr(cell_pls, 'air') or hasattr(cell_mns, 'air'):
        return cal_heat_transfer_byCell( cell_mns, cell_pls )
    else:
        raise NameError('Cellクラスの設定が適切ではありません。')

#### 水蒸気移動

In [31]:
def cal_jv( cell_mns, cell_pls, nx_dir ):
    # 両方のセルがともに多孔質材料である場合
    if hasattr(cell_mns, 'material') and hasattr(cell_pls, 'material'):
        return cal_vapour_permeance_pressure_byCell( cell_mns, cell_pls, nx_dir )
    # 一方が空気境界である場合
    elif hasattr(cell_pls, 'air') or hasattr(cell_mns, 'air'):
        return cal_vapour_transfer_pressure_byCell( cell_mns, cell_pls, nx )
    else:
        raise NameError('Cellクラスの設定が適切ではありません。')

#### 液水移動

In [32]:
def cal_jl( cell_mns, cell_pls, nx_dir ):
    # 両方のセルがともに多孔質材料である場合
    if hasattr(cell_mns, 'material') and hasattr(cell_pls, 'material'):
        return cal_liquid_conduction_potential_byCell( cell_mns, cell_pls, nx_dir )
    # 一方が空気境界である場合
    elif hasattr(cell_pls, 'air') or hasattr(cell_mns, 'air'):
        return 0.0
    else:
        raise NameError('Cellクラスの設定が適切ではありません。')


<br />
<br />

## 4. クラスを用いた流量計算

3節までの計算を流用し、流量をクラスによって管理する方法について示す。
流量をクラスで管理する場合、第三種境界条件における流量計算結果のみならず第二種境界条件を考慮する必要がある。

- 第二種境界条件；一定値を与える方法  
- 第三種境界条件；両端のセル情報から流量を計算する方法  

特に外壁表面などでは第二種と第三種境界条件を組み合わせた条件などが存在する。（日射や降雨が当たる場合など）  
そのため流量の管理モジュールは以下の三種類を有する必要がある。

- 第二種境界条件
- 第三種境界条件（流量計算）  
- 第二種＋第三種境界条件  

クラス化して管理するとmainプログラムが整理され見やすくなる分、計算方法がカプセル化され分かりづらくなるため使用には注意が必要である。


### 第二種境界条件  

第二種境界条件のクラスではそれぞれ熱流・水蒸気流・液水流を保持するだけのクラスとする。


In [23]:
class NeumannBoundaryCondition():
    
    def __init__( self ):
        
        self.qs = 0.0
        self.jv = 0.0
        self.jl = 0.0
        
    # qs
    
    def set_qs(self, qs):
        self.qs = qs

    # jv
    
    def set_jv(self, jv):
        self.jv = jv
    
    # jl
    
    def set_jl(self, jl):
        self.jl = jl
    

### 第三種境界条件
第三種境界条件では上述の3節までの式を用い流量をプロパティ化した。

In [24]:
class RobinBoundaryCondition():
    
    def __init__( self, cell_mns, cell_pls, nx ):
        
        self.cell_mns = cell_mns
        self.cell_pls = cell_pls
        self.nx  = nx
        
    ################################
    # 熱流計算
    
    def cal_heat_conduction( self ):
        return cal_qs(self.cell_mns, self.cell_pls)
    
    qs = property(cal_heat_conduction)
    
    ################################
    # 水蒸気流計算（水蒸気圧差）
    
    def cal_vapour_permeance( self ):
        return cal_jv( self.cell_mns, self.cell_pls, self.nx )
    
    jv = property(cal_vapour_permeance)
    
    ################################
    # 液水流計算（水分化学ポテンシャル差）
    
    def cal_liquid_conduction( self ):
        return cal_jl( self.cell_mns, self.cell_pls, self.nx )
    
    jl = property(cal_liquid_conduction)
    

### 第二種 + 第三種境界条件  

熱伝達＋日射というような形の境界条件の場合こちらを用いる。（等価外気温の形を用いても良いが）  

基本構造は第二種と第三種のクラスの組み合わせである。

In [25]:
class CombinationBoundaryCondition():
    
    def __init__( self, cell_mns, cell_pls, nx ):
        self.neumann = NeumannBoundaryCondition()
        self.robin   = RobinBoundaryCondition(cell_mns, cell_pls, nx)
        
    # Neumann qs
    def set_qs(self, qs):
        self.neumann.set_qs(qs)

    # Neumann jv
    def set_jv(self, jv):
        self.neumann.set_jv(jv)
    
    # Neumann jl
    def set_jl(self, jl):
        self.neumann.set_jl(jl)
    
    ################################
    # 熱流計算
    
    def sum_heat_conduction( self ):
        return self.neumann.qs + self.robin.qs
    
    qs = property(sum_heat_conduction)
    
    # 水蒸気流計算（水蒸気圧差）
    
    def sum_vapour_permeance( self ):
        return self.neumann.jv + self.robin.jv
    
    jv = property(sum_vapour_permeance)
    
    # 液水流計算（水分化学ポテンシャル差）
    
    def sum_liquid_conduction( self ):
        return self.neumann.jl + self.robin.jl
        
    jl = property(sum_liquid_conduction)
    